In [21]:
from datetime import datetime
from dateutil.parser import parse
from openai import OpenAI
import json

In [22]:
# 讀取環境變數(OpenAI API Key)
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
# 初始化 client
client = OpenAI()

In [24]:
# 定義 function schema
functions = [
    {
        "name": "get_current_time",
        "description": "取得目前時間（無需輸入參數）",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "calculate_time_difference",
        "description": "計算兩個時間之間已過或剩餘多久，輸入格式為 'YYYY-MM-DD HH:MM:SS'",
        "parameters": {
            "type": "object",
            "properties": {
                "start_time": {
                    "type": "string",
                    "description": "計算時間差的起始時間，格式為 'YYYY-MM-DD HH:MM:SS'"
                },
                "end_time": {
                    "type": "string",
                    "description": "計算時間差的結束時間，格式為 'YYYY-MM-DD HH:MM:SS'"
                }
            },
            "required": ["start_time", "end_time"]
        }
    }
]

In [25]:
# 本地端 function mapping
def get_current_time() -> str:
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

def calculate_time_difference(start_time: str, end_time: str) -> str:
    try:
        start = parse(start_time)
        end = parse(end_time)
        if start >= end:
            delta = start - end
            prefix = "已過"
        else:
            delta = end - start
            prefix = "還有"
        days = delta.days
        hours = delta.seconds // 3600
        minutes = (delta.seconds % 3600) // 60
        return f"{prefix} {days} 天 {hours} 小時 {minutes} 分鐘"
    except Exception as e:
        return f"發生錯誤，請輸入正確格式：{str(e)}"

In [26]:
# 使用者輸入
user_query = "幫我計算 現在時間 與 2025-08-01 08:00:00 之間還有多久"
# user_query = "幫我計算 2025-08-01 07:00:00 與 2025-08-01 08:00:00 之間還有多久"

In [33]:
# 初始化對話
messages = [
    {"role": "system", "content": "你是會呼叫本地端函式幫助使用者完成需求的 AI 助手。"},
    {"role": "user", "content": user_query}
]

step = 0

while True:
    
    step += 1
    print(f"\n========== 🪐 Step {step}: 發送請求給 GPT 判斷要呼叫哪些工具 ==========")

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=[{"type": "function", "function": fn} for fn in functions],
        tool_choice="auto"
    )

    message = response.choices[0].message

    if message.tool_calls:
        print(f"\n✅ GPT 決定呼叫以下 Tool Calls：")
        results = []
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            print(f"🔸 Tool Call ID: {tool_call.id}")
            print(f"   Function: {function_name}")
            print(f"   Arguments: {arguments}")

            # 執行對應本地端函式
            if function_name == "get_current_time":
                result = get_current_time()
            elif function_name == "calculate_time_difference":
                result = calculate_time_difference(arguments["start_time"], arguments["end_time"])
            else:
                result = "未知的函式"

            print(f"🔹 執行結果: {result}")

            results.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })

        # 將 GPT 回傳含 tool_calls 的 message 與 tool 回傳結果放入上下文
        messages.append(message)
        messages.extend(results)

        # 印出當前完整上下文
        print("\n📄 當前完整 messages 上下文：")
        formatted_msgs = []
        for m in messages:
            role = m.role if hasattr(m, "role") else m["role"]
            content = m.content if hasattr(m, "content") else m["content"]
            formatted_msgs.append({"role": role, "content": content})
        print(json.dumps(formatted_msgs, indent=2, ensure_ascii=False))

    else:
        print(f"\n✅ GPT 已回傳最終文字回覆：")
        print(message.content)
        break


========== 🪐 Step 1: 發送請求給 GPT 判斷要呼叫哪些工具 ==========

✅ GPT 決定呼叫以下 Tool Calls：
🔸 Tool Call ID: call_kgQ8Vg0uudWIcrbAme9QG1F8
   Function: get_current_time
   Arguments: {}
🔹 執行結果: 2025-07-09 22:15:00

📄 當前完整 messages 上下文：
[
  {
    "role": "system",
    "content": "你是會呼叫本地端函式幫助使用者完成需求的 AI 助手。"
  },
  {
    "role": "user",
    "content": "幫我計算 現在時間 與 2025-08-01 08:00:00 之間還有多久"
  },
  {
    "role": "assistant",
    "content": null
  },
  {
    "role": "tool",
    "content": "2025-07-09 22:15:00"
  }
]

========== 🪐 Step 2: 發送請求給 GPT 判斷要呼叫哪些工具 ==========

✅ GPT 決定呼叫以下 Tool Calls：
🔸 Tool Call ID: call_Dm8FThbyWc4tkYXdJBInBscX
   Function: calculate_time_difference
   Arguments: {'start_time': '2025-07-09 22:15:00', 'end_time': '2025-08-01 08:00:00'}
🔹 執行結果: 還有 22 天 9 小時 45 分鐘

📄 當前完整 messages 上下文：
[
  {
    "role": "system",
    "content": "你是會呼叫本地端函式幫助使用者完成需求的 AI 助手。"
  },
  {
    "role": "user",
    "content": "幫我計算 現在時間 與 2025-08-01 08:00:00 之間還有多久"
  },
  {
    "role": "assistant",
  